<a href="https://colab.research.google.com/github/DenisSurovin/statistic-test/blob/main/ANOVA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Задача: определить влияние выбранной диеты на похудение за 6 недель. Всего есть 3 вида диеты. Применяется дисперсионный факторный анализ (ANOVA). Фактор - выбранная диета - имеет 3 уровня. Случайная величина, на которую влияет этот фактор, разница в весе спустя 6 недель. Уровень значимости везде далее предполагается равным α = 0.05.

Подключим необходимые библиотеки.

In [49]:
import pandas as pd
import numpy as np
from scipy import stats

Подключим датасет foodDiet.csv

In [50]:
df = pd.read_csv("https://raw.githubusercontent.com/DenisSurovin/statistic-test/refs/heads/main/foodDiet.csv")

In [51]:
df.head()

,Person,gender,Age,Height,pre.weight,Diet,weight6weeks
0,25,0,41,171,60,2,60.0
1,26,0,32,174,103,2,103.0
2,1,0,22,159,58,1,54.2
3,2,0,46,192,60,1,54.0
4,3,0,55,170,64,1,63.3


Тип диеты указан в столбце Diet. Вес человека в начале диеты - в столбце pre.weight, после - в weight6weeks. Другие факторы, представленные в таблице (пол, возраст, рост) в данном анализе не учитываются.

In [52]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 78 entries, 0 to 77
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Person        78 non-null     int64  
 1   gender        78 non-null     int64  
 2   Age           78 non-null     int64  
 3   Height        78 non-null     int64  
 4   pre.weight    78 non-null     int64  
 5   Diet          78 non-null     int64  
 6   weight6weeks  78 non-null     float64
dtypes: float64(1), int64(6)
memory usage: 4.4 KB


Количество человек, использующие тип диеты 1, 2, 3:

In [53]:
df.Diet.value_counts()

,count
Diet,
2,27
3,27
1,24


Среднее значение потери веса в каждой группе:

In [54]:
mean_loss = [0, 0, 0]
for i in [1, 2, 3]:
  mean_loss[i - 1] = (df[df['Diet'] == i]['weight6weeks'] - df[df['Diet'] == i]['pre.weight']).mean()
  print('Diet', i, 'mean =', round(mean_loss[i - 1], 2))
  #print('Diet', i, 'std =', round((df[df['Diet'] == i]['weight6weeks'] - df[df['Diet'] == i]['pre.weight']).std(), 2))

Diet 1 mean = -3.3
Diet 2 mean = -3.03
Diet 3 mean = -5.15


Необходимо проверить условия применимости ANOVA.
Для проверки гипотезы о нормальности распределения данных в каждой группе используется тест Шапиро-Уилка:

In [55]:
for i in [1, 2, 3]:
  print('Shapiro-Wilk test for Diet', i, '=', stats.shapiro(df[df['Diet'] == i]['weight6weeks'] - df[df['Diet'] == i]['pre.weight'] - mean_loss[i - 1]))

Shapiro-Wilk test for Diet 1 = ShapiroResult(statistic=np.float64(0.9255313246138968), pvalue=np.float64(0.07748633505906967))
Shapiro-Wilk test for Diet 2 = ShapiroResult(statistic=np.float64(0.9855912340453256), pvalue=np.float64(0.9611748496261021))
Shapiro-Wilk test for Diet 3 = ShapiroResult(statistic=np.float64(0.9601254566120733), pvalue=np.float64(0.3720400277004393))


В каждой группе p-value получилось больше заданного уровня значимости, поэтому нет статистически значимых оснований для отвержения нулевой гипотезы о том, что данные в каждой группе распределены нормально.

Далее необходимо проверить условие об однородности дисперсий в каждой группе. Для этого будем использовать тест Бартлетта.

In [56]:
stats.bartlett(df[df['Diet'] == 1]['weight6weeks'] - df[df['Diet'] == 1]['pre.weight'],
               df[df['Diet'] == 2]['weight6weeks'] - df[df['Diet'] == 2]['pre.weight'],
               df[df['Diet'] == 3]['weight6weeks'] - df[df['Diet'] == 3]['pre.weight'])

BartlettResult(statistic=np.float64(0.337454630479139), pvalue=np.float64(0.8447392194944549))

p-value оказалось больше уровня значимости, поэтому нет статистически значимых оснований считать, что дисперсии в группах различны.

Поскольку условия для применения ANOVA выполняются, проведем тест о равенстве средних в каждой группе.

In [57]:
stats.f_oneway(df[df['Diet'] == 1]['weight6weeks'] - df[df['Diet'] == 1]['pre.weight'],
               df[df['Diet'] == 2]['weight6weeks'] - df[df['Diet'] == 2]['pre.weight'],
               df[df['Diet'] == 3]['weight6weeks'] - df[df['Diet'] == 3]['pre.weight'])

F_onewayResult(statistic=np.float64(6.197447453165349), pvalue=np.float64(0.0032290142385893524))

Так как p-value оказался меньше уровня значимости, есть статистически значимые основания отвергнуть нулевую гипотезу о равенстве средних в группах. Для того, чтобы определить, в каких именно группах средние различаются, проведем попарные двухвыборочные t-тесты.

In [58]:
stats.ttest_ind(df[df['Diet'] == 1]['weight6weeks'] - df[df['Diet'] == 1]['pre.weight'],
               df[df['Diet'] == 2]['weight6weeks'] - df[df['Diet'] == 2]['pre.weight'])

TtestResult(statistic=np.float64(-0.40797824323257154), pvalue=np.float64(0.6850668861405854), df=np.float64(49.0))

In [59]:
stats.ttest_ind(df[df['Diet'] == 1]['weight6weeks'] - df[df['Diet'] == 1]['pre.weight'],
               df[df['Diet'] == 3]['weight6weeks'] - df[df['Diet'] == 3]['pre.weight'])

TtestResult(statistic=np.float64(2.834783037306771), pvalue=np.float64(0.006644381649741192), df=np.float64(49.0))

In [60]:
stats.ttest_ind(df[df['Diet'] == 3]['weight6weeks'] - df[df['Diet'] == 3]['pre.weight'],
               df[df['Diet'] == 2]['weight6weeks'] - df[df['Diet'] == 2]['pre.weight'])

TtestResult(statistic=np.float64(-3.1693496673045676), pvalue=np.float64(0.0025599026452984013), df=np.float64(52.0))

Из значений p-value можно сделать вывод о том, что нет статистически значимых основаниях считать, что средние в 1 и 2 группах различны. В то же время, среднее в 3 группе отличается от них.

In [61]:
for i in range(3):
  print('Diet', i + 1, 'mean weight loss =', mean_loss[i])

Diet 1 mean weight loss = -3.2999999999999994
Diet 2 mean weight loss = -3.025925925925926
Diet 3 mean weight loss = -5.148148148148147


Таким образом, учитывая, что средняя потеря веса в группе 3 больше, чем в 1 и 2 группах, можно сделать вывод о том, что диета 3 наиболее эффективна в похудении.